In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [ ]:
file = './data/clothing-catalog.csv'
loader = CSVLoader(file_path=file)

In [ ]:
#!pip install docarray

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
                deployment="text-embedding-ada-002",
                model="text-embedding-ada-002",
                openai_api_base=os.environ["OPENAI_BASE_URL"],
                openai_api_version="2023-06-01-preview",
                openai_api_type="azure",
                chunk_size = 1
            )

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

In [ ]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [ ]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_base=os.environ['OPENAI_BASE_URL'],
    deployment_name="chat",
    openai_api_version="2023-06-01-preview",
    openai_api_key=os.environ['OPENAI_API_KEY'],
    temperature=0.0)

In [ ]:
response = index.query(query, llm=llm)

In [ ]:
display(Markdown(response))

In [ ]:
loader = CSVLoader(file_path=file)
docs = loader.load()
docs[0]

In [ ]:
# from langchain.embeddings import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings()
# Will use the embeddings created above !!

In [ ]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [ ]:
query = "Please suggest a shirt with sunblocking"

In [ ]:
docs = db.similarity_search(query)

In [ ]:
docs[0]

In [ ]:
retriever = db.as_retriever()

In [ ]:
# llm = ChatOpenAI(temperature = 0.0)
# Will use the above LLM

In [ ]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [ ]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [ ]:
display(Markdown(response))

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [ ]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [ ]:
response = qa_stuff.run(query)

In [ ]:
display(Markdown(response))